In [1]:
# Import the data
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, balanced_accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
#from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import StandardScaler, MinMaxScaler
#import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
#from sklearn.linear_model import LogisticRegression
#from sklearn.metrics import confusion_matrix, classification_report, balanced_accuracy_score, roc_auc_score
#from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [2]:
df = pd.read_csv("Resources/bank/bank-full.csv")
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [3]:
df.shape

(45211, 17)

In [4]:
df.info() #check types and for null elements

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [5]:
# Drop rows with null values
df_clean = df.dropna().copy()

# Verify changes with the info method
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [6]:

# Convert y to numeric
df_clean['y'] = pd.get_dummies(df_clean['y'], drop_first = True, dtype=int)

#df_clean.loc[df_clean['poutcome']=='success'].head()
df_clean.loc[df_clean['y']==1]

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
83,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,1
86,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,1
87,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,1
129,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,1
168,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45204,73,retired,married,secondary,no,2850,no,no,cellular,17,nov,300,1,40,8,failure,1
45205,25,technician,single,secondary,no,505,no,yes,cellular,17,nov,386,2,-1,0,unknown,1
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,1
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,1


In [7]:
df['job'].unique()

array(['management', 'technician', 'entrepreneur', 'blue-collar',
       'unknown', 'retired', 'admin.', 'services', 'self-employed',
       'unemployed', 'housemaid', 'student'], dtype=object)

In [8]:
df['marital'].unique()

array(['married', 'single', 'divorced'], dtype=object)

In [9]:
df['education'].unique()

array(['tertiary', 'secondary', 'unknown', 'primary'], dtype=object)

In [10]:
df['education'].head()

0     tertiary
1    secondary
2    secondary
3      unknown
4      unknown
Name: education, dtype: object

In [37]:
#enc_Ord = OrdinalEncoder() #not ranked. single people matter <3
enc_Ord = OrdinalEncoder() #not ranked. single people matter <3
enc_One = OneHotEncoder()
#onehot for marital, job
#df[["Sex","Blood", "Study"]] = enc.fit_transform(df[["Sex","Blood", "Study"]])
#one_array = enc_One.fit_transform(df[['job', 'marital']])
#one_array.head()
#pd.DataFrame(one_array, index=df.index)
#df[['education']] = enc_Ord.fit_transform(df[['education']])
#df[['job', 'marital','education']].head()
# Set up the OneHotEncoder so it will transform to Pandas
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
ohe.set_output(transform="pandas")

# Fit and transform the OneHotEncoder to the columns to encode
#car_data_ohe = ohe.fit_transform(car_data[columns_to_encode])
df_data_ohe = ohe.fit_transform(df[['job', 'marital']])
df_data_ohe.head()
#car_data_ohe.head()
#df_ohe = ohe.fit_transform(df[['job']]).values.reshape(-1, 1)
df_ohe[:10]

array([[0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [38]:
enc_Ord = OrdinalEncoder() #not ranked. single people matter <3
df[['education']] = enc_Ord.fit_transform(df[['education']])
df['education'].head()

0    2.0
1    1.0
2    1.0
3    3.0
4    3.0
Name: education, dtype: float64

In [12]:
# Create an instance of OneHotEncoder()
enc = OneHotEncoder(handle_unknown='ignore')

# Fit the encoder to the data
#enc.fit(car_data[columns_to_encode])
enc.fit(df[['job', 'marital']])

# Transform the data
car_data_ohe = enc.transform(df[['job', 'marital']])
car_data_ohe


<45211x15 sparse matrix of type '<class 'numpy.float64'>'
	with 90422 stored elements in Compressed Sparse Row format>

In [14]:
#~~~~~~~~~~~~~~~~~
#plot outliers in age, bank balance
#plot^^
#~~~~~~~~~~~~~~~~~

In [15]:
# Drop all non-numeric columns
df_clean = df_clean.select_dtypes(include='number')

# Verify changes with the info method
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   age       45211 non-null  int64
 1   balance   45211 non-null  int64
 2   day       45211 non-null  int64
 3   duration  45211 non-null  int64
 4   campaign  45211 non-null  int64
 5   pdays     45211 non-null  int64
 6   previous  45211 non-null  int64
 7   y         45211 non-null  int32
dtypes: int32(1), int64(7)
memory usage: 2.6 MB


In [16]:
# Split training and testing sets
# Create the features DataFrame, X
X = df_clean.copy()
X = X.drop(columns='y')

# Create the target DataFrame, y
y = df_clean['y']

In [17]:
from sklearn.linear_model import LogisticRegression


# Create a `LogisticRegression` function and assign it 
# to a variable named `logistic_regression_model`.
logistic_regression_model = LogisticRegression()

In [18]:
# Define target vector
#y_array = df_clean["y"].values.reshape(-1, 1)
#df_clean.loc[df_clean['poutcome']=='success'].head()
#y.loc[y==1]
#y_array = y.values.reshape(-1, 1)
#y_array[:5]
y_array = y.values
y_array[:5]

array([0, 0, 0, 0, 0])

In [26]:
# Use train_test_split to separate the data
#X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y_array, random_state=34)

In [27]:
X_train

,age,balance,day,duration,campaign,pdays,previous
15255,49,0,17,51,1,-1,0
37448,58,4152,13,76,7,13,6
40675,38,64,6,789,1,-1,0
41644,50,0,29,159,1,120,2
13140,38,338,8,191,1,-1,0
...,...,...,...,...,...,...,...
28406,28,959,29,330,2,-1,0
22377,44,267,22,520,2,-1,0
43498,22,3472,19,529,1,181,4
38010,35,650,14,241,2,349,2


In [28]:
# Scaling the X data by using StandardScaler()
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_train_scaled

array([[ 0.75831894, -0.44147479,  0.14119639, ..., -0.57135633,
        -0.41212268, -0.24074168],
       [ 1.60588178,  0.88965466, -0.33913341, ...,  1.36732347,
        -0.272179  ,  2.20891392],
       [-0.27759119, -0.42095642, -1.17971057, ..., -0.57135633,
        -0.41212268, -0.24074168],
       ...,
       [-1.78436956,  0.67164694,  0.3813613 , ..., -0.57135633,
         1.40714514,  1.39236205],
       [-0.56011213, -0.23308506, -0.21905096, ..., -0.24824303,
         3.08646928,  0.57581018],
       [ 1.70005543,  1.78829527,  0.50144375, ...,  2.33666337,
        -0.41212268, -0.24074168]])

In [29]:
# Scaling the X data by using StandardScaler()
scaler = StandardScaler().fit(X_test)
X_test_scaled = scaler.transform(X_test)
X_test_scaled

array([[ 1.70440129, -0.35561854,  0.99282268, ..., -0.563368  ,
        -0.40945353, -0.3075946 ],
       [-0.46163802,  0.0993152 ,  0.63165779, ..., -0.563368  ,
        -0.40945353, -0.3075946 ],
       [ 1.04517194, -0.35739841, -1.41494327, ..., -0.24153698,
        -0.40945353, -0.3075946 ],
       ...,
       [ 0.29176696, -0.44425587,  0.51126949, ...,  0.08029403,
        -0.40945353, -0.3075946 ],
       [-0.17911115, -0.22711222, -0.090672  , ..., -0.563368  ,
         3.19650863,  0.80621023],
       [ 0.4801182 ,  0.9490232 , -0.33144859, ..., -0.24153698,
        -0.40945353, -0.3075946 ]])

In [30]:
y_train[:5]

array([0, 0, 0, 1, 0])

In [31]:
# Fit the model
#logistic_regression_model.fit(X_train_scaled, y_train)
#logistic_regression_model.fit(X_train, y_train)
logistic_regression_model.fit(X_train_scaled, y_train)

LogisticRegression()

In [32]:
# Score the model
print(f"Training Data Score: {logistic_regression_model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {logistic_regression_model.score(X_test_scaled, y_test)}")

Training Data Score: 0.887519169517518
Testing Data Score: 0.8932141909227639


In [19]:
#Score the model
#print(f"Training Data Score: {logistic_regression_model.score(X_train, y_train)}")
#print(f"Testing Data Score: {logistic_regression_model.score(X_test, y_test)}")

In [20]:
# Create a Logistic Regression Model
#classifier = LogisticRegression()

# Fit (train) or model using the training data
#classifier.fit(X, y)

# Calculate the accuracy of the model
#classifier.score(X, y)

C:\Users\symia\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8823269188232692

In [22]:
# Make predictions on the test data
#predictions = classifier.predict(X)

# Create a confusion matrix
#print(confusion_matrix(y, predictions, labels = [1,0]))

[[  73  448]
 [  84 3916]]
